# Set Up for Project Imports


In [6]:
import sys
from pathlib import Path

In [7]:
source_directory = Path.cwd()
ROOT = source_directory.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [8]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [9]:
from data.api import UcIrvineAPI, UcIrvineDatasetIDs, BureauEconomicAnalysisAPI
import json
import pandas

In [11]:
pandas.set_option('display.max_colwidth', None)  # show all text in cells
pandas.options.mode.copy_on_write = True

# UcIrvine Data

In [12]:
uci_df = UcIrvineAPI.fetch_dataset(repo_id=UcIrvineDatasetIDs.Apartment_For_Rent_Classified.value)

/Users/Isaia/PycharmProjects/CS-577-07-Data-Science-Fall-2025-Project/.venv/lib/python3.12/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (0,5,6,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


# Bureau of Economic Data

In [13]:
bea_df = BureauEconomicAnalysisAPI.fetch_dataset('Regional', GeoFips='STATE', TableName='SARPP', Year='2019',
                                                 LineCode='1')